1. Introduction:

Human Resource (HR) Data of a Multi-national Corporation (MNC)

This dataset contains HR information for employees of a multinational corporation (MNC). It includes 2 Million (20 Lakhs) employee records with details about personal identifiers, job-related attributes, performance, employment status, and salary information.
The dataset can be used for HR analytics, including workforce distribution, attrition analysis, salary trends, and performance evaluation.


In [ ]:
2. ASK:

What are the purposes of this analysis:
- Workforce Distribution: Analyzing the demographics and structure of the workforce across different departments, locations, or roles. This could help management understand the composition of their staff and identify any imbalances.

- Attrition Analysis: Studying employee turnover to understand why employees are leaving the company. By analyzing data on terminated employees, an organization can identify trends and potential root causes of attrition, such as low job satisfaction or poor management.

- Salary Trends: Examining salary data to identify patterns, ensure fair compensation, and detect potential pay gaps. This can be used for benchmarking salaries against industry standards or for making informed decisions on compensation adjustments.

- Performance Evaluation: Assessing employee performance to identify high-achievers, pinpoint areas for improvement, and link performance to other factors like salary or retention.

And they will be achieved by answering these questions:

Q.1) What is the distribution of Employee Status (Active, Resigned, Retired, Terminated) ?

Q.2) What is the distribution of work modes (On-site, Remote) ?

Q.3) How many employees are there in each department ?

Q.4) What is the average salary by Department ?

Q.5) Which job title has the highest average salary ?

Q.6) What is the average salary in different Departments based on Job Title ?

Q.7) How many employees Resigned & Terminated in each department ?

Q.8) How does salary vary with years of experience ?

Q.9) What is the average performance rating by department ?

Q.10) Which Country have the highest concentration of employees ?

Q.11) Is there a correlation between performance rating and salary ?

Q.12) How has the number of hires changed over time (per year) ?

Q.13) Compare salaries of Remote vs. On-site employees — is there a significant difference ?

Q.14) Find the top 10 employees with the highest salary in each department.

Q.15) Identify departments with the highest attrition rate (Resigned %).

3. PREPARE:
Setup library and import dataset

4. PROCESS:

5. ANALYSIS:

6. SHARING & ACT: